In [1]:
import numpy as np
import pandas as pd 
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2, VGG19
from tensorflow.keras.models import Sequential
from keras import regularizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, InputLayer, Reshape, Conv1D, MaxPool1D, SeparableConv2D
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import cross_validate, train_test_split
import matplotlib.pyplot as plt

In [7]:
import shutil
print(os.listdir("archive/cell_images/cell_images"))

base_dir = 'archive/cell_images/cell_images/'
work_dir = 'work/'
#os.mkdir(work_dir)

base_dir_A = 'archive/cell_images/cell_images/Parasitized/' 
base_dir_B = 'archive/cell_images/cell_images/Uninfected/'

work_dir_A = 'work/A/'
#os.mkdir(work_dir_A)
work_dir_B = 'work/B/'
#os.mkdir(work_dir_B)

['Parasitized', 'Uninfected']


In [8]:
train_dir = os.path.join(work_dir, 'train')
#os.mkdir(train_dir)

validation_dir = os.path.join(work_dir, 'validation')
#os.mkdir(validation_dir)

test_dir = os.path.join(work_dir, 'test')
#os.mkdir(test_dir)

print("New directories for train, validation, and test created")
train_pos_dir = os.path.join(train_dir, 'pos')
#os.mkdir(train_pos_dir)
train_neg_dir = os.path.join(train_dir, 'neg')
#os.mkdir(train_neg_dir)

validation_pos_dir = os.path.join(validation_dir, 'pos')
#os.mkdir(validation_pos_dir)
validation_neg_dir = os.path.join(validation_dir, 'neg')
#os.mkdir(validation_neg_dir)

test_pos_dir = os.path.join(test_dir, 'pos')
#os.mkdir(test_pos_dir)
test_neg_dir = os.path.join(test_dir, 'neg')
#os.mkdir(test_neg_dir)

print("Train, Validation, and Test folders made for both A and B datasets")

New directories for train, validation, and test created
Train, Validation, and Test folders made for both A and B datasets


In [9]:
i = 0
      
for filename in os.listdir(base_dir_A): 
    dst ="pos" + str(i) + ".jpg"
    src =base_dir_A + filename 
    dst =work_dir_A + dst 
          
       # rename() function will 
       # rename all the files 
    shutil.copy(src, dst) 
    i += 1


j = 0

for filename in os.listdir(base_dir_B): 
    dst ="neg" + str(j) + ".jpg"
    src =base_dir_B + filename 
    dst =work_dir_B + dst 
          
    # rename() function will 
    # rename all the files 
    shutil.copy(src, dst) 
    j += 1       
        
print("Images for both categories have been copied to working directories, renamed to A & B + num")

Images for both categories have been copied to working directories, renamed to A & B + num


In [10]:
fnames = ['pos{}.jpg'.format(i) for i in range(3000)]
for fname in fnames:
    src = os.path.join(work_dir_A, fname)
    dst = os.path.join(train_pos_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['pos{}.jpg'.format(i) for i in range(3000, 4000)]
for fname in fnames:
    src = os.path.join(work_dir_A, fname)
    dst = os.path.join(validation_pos_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['pos{}.jpg'.format(i) for i in range(4000, 4500)]
for fname in fnames:
    src = os.path.join(work_dir_A, fname)
    dst = os.path.join(test_pos_dir, fname)
    shutil.copyfile(src, dst)

In [11]:
fnames = ['neg{}.jpg'.format(i) for i in range(3000)]
for fname in fnames:
    src = os.path.join(work_dir_B, fname)
    dst = os.path.join(train_neg_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['neg{}.jpg'.format(i) for i in range(3000, 4000)]
for fname in fnames:
    src = os.path.join(work_dir_B, fname)
    dst = os.path.join(validation_neg_dir, fname)
    shutil.copyfile(src, dst)

fnames = ['neg{}.jpg'.format(i) for i in range(4000, 4500)]
for fname in fnames:
    src = os.path.join(work_dir_B, fname)
    dst = os.path.join(test_neg_dir, fname)
    shutil.copyfile(src, dst)
    
print("Train, validation, and test datasets split and ready for use")
print('total training pos images:', len(os.listdir(train_pos_dir)))
print('total training neg images:', len(os.listdir(train_neg_dir)))
print('total validation pos images:', len(os.listdir(validation_pos_dir)))
print('total validation neg images:', len(os.listdir(validation_neg_dir)))
print('total test pos images:', len(os.listdir(test_pos_dir)))
print('total test neg images:', len(os.listdir(test_neg_dir)))

Train, validation, and test datasets split and ready for use
total training pos images: 3000
total training neg images: 3000
total validation pos images: 1000
total validation neg images: 1000
total test pos images: 500
total test neg images: 500


In [12]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.33)

In [13]:
train_generator = train_datagen.flow_from_directory(directory= train_dir,             
                                                     target_size=(128, 128),
                                                     class_mode='binary',
                                                     subset='training',
                                                    shuffle=True,
                                                     batch_size=32
                                 )

valid_generator = train_datagen.flow_from_directory(directory= validation_dir,
                                                      target_size=(128, 128),
                                                     class_mode='binary',
                                                           shuffle = True,
                                                     subset='validation',
                                                     batch_size=32,
                                                    
                                                     )


classes = ['Parasitized', 'Uninfected']

Found 4020 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [14]:
vgg_model = Sequential()
vgg_model.add(VGG19(include_top=False, pooling='avg', weights='imagenet', input_shape=(128, 128, 3), classes=2))
vgg_model.add(Flatten())
vgg_model.add(Dense(256,activation='relu'))
vgg_model.add(Dense(64,activation='relu'))
vgg_model.add(Dense(1,activation = 'sigmoid'))

vgg_model.layers[0].trainable = False

opt = tf.keras.optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999)
vgg_model.compile(optimizer= opt, loss='binary_crossentropy', metrics=['accuracy'])
vgg_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 512)               20024384  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 20,172,225
Trainable params: 147,841
Non-trainable params: 20,024,384
_________________________________________________________________


In [16]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)

In [19]:
vgg_history = vgg_model.fit_generator(train_generator,
                              steps_per_epoch = len(train_generator),
                              epochs=10,
                              validation_steps = len(valid_generator),
                                      validation_data = valid_generator,
                              callbacks = [early_stop],
                                      verbose=1
                                     )

Epoch 1/10


C:\Users\kotla\AppData\Local\Temp\ipykernel_34148\2786986023.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  vgg_history = vgg_model.fit_generator(train_generator,


126/126 [==============================] - 166s 1s/step - loss: 0.2496 - accuracy: 0.9032 - val_loss: 0.1406 - val_accuracy: 0.9606
Epoch 2/10
126/126 [==============================] - 165s 1s/step - loss: 0.2334 - accuracy: 0.9119 - val_loss: 0.1893 - val_accuracy: 0.9364


In [20]:
vgg_model.save('vgg19.h5')

In [34]:
#predict on specific image
from matplotlib import image


#predict on specific image
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

img = image.load_img('archive/cell_images/cell_images/Uninfected/C101P62ThinF_IMG_20150918_151006_cell_17.png', target_size=(128, 128))
# plt.imshow(img)
# plt.show()

img = image.img_to_array(img)
img = img/255.0
img = np.expand_dims(img, axis=0)

prediction = vgg_model.predict(img)
print(prediction)

if prediction > 0.5:
    print("The image classified is Parasitized")
else:
    print("The image classified is Uninfected")


1/1 [==============================] - 0s 79ms/step
[[0.23628019]]
The image classified is Uninfected
